In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
#定義常用函數
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
#準備訓練資料
x_data = np.linspace(-2, 2, 500, dtype = np.double)[:,np.newaxis]
y_data = x_data.copy() + np.random.normal(0, 0.1, x_data.shape)

In [ ]:
#學習速率
learning_rate = 0.001

In [ ]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

In [ ]:
myNN = Network([1,2,3,1])
print(myNN.biases, myNN.weights)

In [ ]:
print(myNN.feedforward(3))